In [2]:
import os 
from pandasai import SmartDataframe
from pandasai.llm import OpenAI
import pandas as pd
import json
from dotenv import load_dotenv
load_dotenv()
llm = OpenAI(model_name='gpt-4-turbo')
import sys
sys.path.append("/Users/suryabhosale/Documents/projects/DORIS/src/POCClinicalTrial")
from utils.config import *

**Testing on MicroData:**

1. Whenever a new document reference is loaded, the runtime has to be reset to create new cache depot

In [3]:
df = pd.DataFrame({
    "country": [
        "United States",
        "United Kingdom",
        "France",
        "Germany",
        "Italy",
        "Spain",
        "Canada",
        "Australia",
        "Japan",
        "China",
    ],
    "gdp": [
        19294482071552,
        2891615567872,
        2411255037952,
        3435817336832,
        1745433788416,
        1181205135360,
        1607402389504,
        1490967855104,
        4380756541440,
        14631844184064,
    ],
    "happiness_index": [6.94, 7.16, 6.66, 7.07, 6.38, 6.4, 7.23, 7.22, 5.87, 5.12],
})

In [4]:
from pandasai import Agent
from pandasai.llm import OpenAI

llm = OpenAI()

agent = Agent(df, config={"llm": llm})
agent.train(docs="Exclude China from the highest GDP")

In [5]:
response = agent.chat("Which country has the lowest GDP?")
print(response)

The country with the lowest GDP is Spain.


______________________________________________________________________________

#### Testing on mock clinical trial data:

In [5]:
from pandasai import SmartDataframe
from pandasai.llm import OpenAI
import pandas as pd
import json
from dotenv import load_dotenv
load_dotenv()
llm = OpenAI(model_name='gpt-4-turbo')
import sys
sys.path.append("/Users/suryabhosale/Documents/projects/DORIS/src/POCClinicalTrial")
from utils.config import *
from pandasai import Agent

In [6]:
df = pd.read_csv('../ct_csv/FinalCTTrialsDF_P1.csv')
df = df[[col for col in df.columns if 'Unnamed' not in col]]
df

,NCT_NUMBER,STUDY_TITLE,STUDY_URL,STUDY_STATUS,CONDITIONS,START_DATE,COMPLETION_DATE,AGE,ELIGIBILITY_CRITERIA,PRIMARY_OUTCOMES,SECONDARY_OUTCOMES,INTERVENTIONS,FACILITY,CITY,STATE,ZIP,COUNTRY
0,NCT03275311,Evolution of the Therapeutic Care in Metastati...,https://www.clinicaltrials.gov/study/NCT03275311,Recruiting,Metastatic Breast Cancer,July 2014,December 2024,18 Years||Adult|Older Adult,Eligibility includes patients 18 years or olde...,The primary outcome measure is to describe the...,Secondary outcomes focus on the impact of ther...,The study does not specify particular interven...,Institut de Cancérologie de l'Ouest - Paul Papin,Angers,NaN,49933,France
1,NCT03275311,Evolution of the Therapeutic Care in Metastati...,https://www.clinicaltrials.gov/study/NCT03275311,Recruiting,Metastatic Breast Cancer,July 2014,December 2024,18 Years||Adult|Older Adult,Eligibility includes patients 18 years or olde...,The primary outcome measure is to describe the...,Secondary outcomes focus on the impact of ther...,The study does not specify particular interven...,Institut Bergonié,Bordeaux,NaN,33076,France
2,NCT03275311,Evolution of the Therapeutic Care in Metastati...,https://www.clinicaltrials.gov/study/NCT03275311,Recruiting,Metastatic Breast Cancer,July 2014,December 2024,18 Years||Adult|Older Adult,Eligibility includes patients 18 years or olde...,The primary outcome measure is to describe the...,Secondary outcomes focus on the impact of ther...,The study does not specify particular interven...,Centre François Baclesse,Caen,NaN,14076,France
3,NCT03275311,Evolution of the Therapeutic Care in Metastati...,https://www.clinicaltrials.gov/study/NCT03275311,Recruiting,Metastatic Breast Cancer,July 2014,December 2024,18 Years||Adult|Older Adult,Eligibility includes patients 18 years or olde...,The primary outcome measure is to describe the...,Secondary outcomes focus on the impact of ther...,The study does not specify particular interven...,Centre Jean Perrin,Clermont-ferrand,NaN,63011,France
4,NCT03275311,Evolution of the Therapeutic Care in Metastati...,https://www.clinicaltrials.gov/study/NCT03275311,Recruiting,Metastatic Breast Cancer,July 2014,December 2024,18 Years||Adult|Older Adult,Eligibility includes patients 18 years or olde...,The primary outcome measure is to describe the...,Secondary outcomes focus on the impact of ther...,The study does not specify particular interven...,Centre Georges-François Leclerc,Dijon,NaN,21079,France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17926,NCT05905939,HER2(Human Epidermal Growth Factor Receptor 2)...,https://www.clinicaltrials.gov/study/NCT05905939,Recruiting,Breast Cancer,"May 31, 2023","May 31, 2024",||Child|Adult|Older Adult,Eligibility includes patients with HER2-positi...,The primary outcomes aim to describe the clini...,Secondary outcomes focus on describing diagnos...,"The study does not specify interventions, as i...",Research Site,Ulan-Ude,NaN,NaN,Russian Federation
17927,NCT05905939,HER2(Human Epidermal Growth Factor Receptor 2)...,https://www.clinicaltrials.gov/study/NCT05905939,Recruiting,Breast Cancer,"May 31, 2023","May 31, 2024",||Child|Adult|Older Adult,Eligibility includes patients with HER2-positi...,The primary outcomes aim to describe the clini...,Secondary outcomes focus on describing diagnos...,"The study does not specify interventions, as i...",Research Site,Vladivostok,NaN,NaN,Russian Federation
17928,NCT05905939,HER2(Human Epidermal Growth Factor Receptor 2)...,https://www.clinicaltrials.gov/study/NCT05905939,Recruiting,Breast Cancer,"May 31, 2023","May 31, 2024",||Child|Adult|Older Adult,Eligibility includes patients with HER2-positi...,The primary outcomes aim to describe the clini...,Secondary outcomes focus on describing diagnos...,"The study does not specify interventions, as i...",Research Site,Volgograd,NaN,NaN,Russian Federation
17929,NCT05905939,HER2(Human Epidermal Growth Factor Receptor 2)...,https://www.clinicaltrials.gov/study/

In [7]:
agent = Agent(df, config={"llm": llm})
agent.train(docs="You are a search assistant for clinical trials. Priorotise CITY, STATE, COUNTRY to filter on location and then search columns CONDITIONS, ELIGIBILITY_CRITERIA, INTERVENTIONS, PRIMARY_OUTCOMES, SECONDARY_OUTCOMES to match medical conditions in of clinical trials in that area before returning dataframe")

In [8]:

agent.chat('Generate a list of clinical trials for a patient with newly diagnosed stage IV NSCLC for first line treatment options in the Chicago area?')

'No clinical trials found for a patient with newly diagnosed stage IV NSCLC for first line treatment options in the Chicago area.'

______________________________________________________________________

Testing Concurrency:

In [6]:
import time
import concurrent
from concurrent.futures import ThreadPoolExecutor

def process_data(num):
    time.sleep(1)
    print("Processed:", num)

numbers = [1, 2, 3, 4, 5, 6, 7, 8]

with ThreadPoolExecutor(max_workers=3) as executor:
    future_to_number = {executor.submit(process_data, num): num for num in numbers}
    for future in concurrent.futures.as_completed(future_to_number):
        num = future_to_number[future]
        print("Completed processing:", num)
    

Processed:Processed: 2
Processed: 3
Completed processing: 2
Completed processing: 3
 1
Completed processing: 1
Processed:Processed: 5
Completed processing: 5
 4
Processed: 6
Completed processing: 4
Completed processing: 6
Processed: 8
Completed processing: 8
Processed: 7
Completed processing: 7


In [10]:
import threading
import time

# Define the first function
def function1():
        time.sleep(1)
        print("Function 1 is running")

# Define the second function
def function2():
        time.sleep(3)
        print("Function 2 is running")

# Create threads for each function
thread1 = threading.Thread(target=function1)
thread2 = threading.Thread(target=function2)

# Start both threads
thread1.start()
thread2.start()

# Wait for both threads to finish
thread1.join()
thread2.join()

print("Both functions have finished")

Function 1 is running
Function 2 is running
Both functions have finished


### Playground

In [10]:
import pandas as pd 
df = pd.read_csv('../ct_csv/FinalCTTrialsDF_P1.csv')
[i for i in df['CITY'].unique()]
df[df['STATE'] == 'Illinois']['CITY'].unique()

array(['Skokie', 'Chicago', 'Alton', 'Aurora', 'Barrington',
       'Bloomington', 'Canton', 'Carbondale', 'Carterville', 'Carthage',
       'Centralia', 'Crystal Lake', 'Danville', 'Decatur', 'Dixon',
       'Downers Grove', 'Effingham', 'Elgin', 'Eureka', 'Evanston',
       'Galesburg', 'Glenview', 'Hazel Crest', 'Highland Park', 'Hines',
       'Joliet', 'Kewanee', 'Libertyville', 'Macomb', 'Mattoon',
       'Maywood', 'Moline', 'Mount Vernon', 'New Lenox', "O'Fallon",
       'Oak Lawn', 'Orland Park', 'Ottawa', 'Park Ridge', 'Pekin',
       'Peoria', 'Peru', 'Princeton', 'Rockford', 'Shiloh', 'Silvis',
       'Springfield', 'Swansea', 'Urbana', 'Washington', 'Yorkville',
       'Arlington Heights', 'Burr Ridge', 'Elmhurst', 'Freeport',
       'Hinsdale', 'Homer Glen', 'Lake Forest', 'Melrose Park',
       'Naperville', 'Plainfield', 'DeKalb', 'Geneva', 'Oak Park',
       'Warrenville', 'Winfield', 'Evergreen Park', 'Rolling Meadows',
       'Harvey', 'Normal', 'Grayslake', 'River F

Testing langchain agents:

In [32]:
from langchain.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
# from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [57]:
# from langchain.agents import AgentType, create_openai_tools_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
import pandas as pd

In [58]:
from langchain_community.agent_toolkits import create_openapi_agent